In [10]:
import cv2
import numpy as np
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

def run():
    audio_devices = AudioUtilities.GetSpeakers()
    volume_interface = audio_devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume_control = cast(volume_interface, POINTER(IAudioEndpointVolume))
    volume_range = volume_control.GetVolumeRange()
    min_volume, max_volume, _ = volume_range

    # Initialize the hand tracking model
    mp_hand = mp.solutions.hands
    hand_tracker = mp_hand.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.75, min_tracking_confidence=0.75)
    drawing_utils = mp.solutions.drawing_utils
    
    camera = cv2.VideoCapture(0)
    
    try:
        while camera.isOpened():
            success, image = camera.read()
            if not success:
                break
            image = cv2.flip(image, 1)
            
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            result = hand_tracker.process(rgb_image)

            # Extract landmarks for both hands
            left_landmarks, right_landmarks = extract_hand_landmarks(image, result, drawing_utils, mp_hand)

            # Adjust brightness using the left hand
            if left_landmarks:
                left_distance = calculate_distance(image, left_landmarks)
                brightness = np.interp(left_distance, [50, 220], [0, 100])
                sbc.set_brightness(int(brightness))
                print(f"Brightness set to: {int(brightness)}")

            # Adjust volume using the right hand
            if right_landmarks:
                right_distance = calculate_distance(image, right_landmarks)
                volume_level = np.interp(right_distance, [50, 220], [min_volume, max_volume])
                volume_control.SetMasterVolumeLevel(volume_level, None)
                print(f"Volume set to: {volume_level}")

            cv2.imshow('Camera Feed', image)
            if cv2.waitKey(1) & 0xFF == 27:
                break
    finally:
        camera.release()
        cv2.destroyAllWindows()

def extract_hand_landmarks(image, result, drawing_utils, mp_hand):
    left_landmarks = []
    right_landmarks = []

    if result.multi_hand_landmarks:
        for index, hand_landmarks in enumerate(result.multi_hand_landmarks):
            hand_label = result.multi_handedness[index].classification[0].label

            for landmark_index, landmark in enumerate(hand_landmarks.landmark):
                height, width, _ = image.shape
                x, y = int(landmark.x * width), int(landmark.y * height)

                if landmark_index == 4 or landmark_index == 8:
                    point = [landmark_index, x, y]

                    if hand_label == 'Left':
                        left_landmarks.append(point)
                    elif hand_label == 'Right':
                        right_landmarks.append(point)

            drawing_utils.draw_landmarks(image, hand_landmarks, mp_hand.HAND_CONNECTIONS)

    return left_landmarks, right_landmarks

def calculate_distance(image, landmarks):
    if len(landmarks) < 2:
        return 0
    (x1, y1), (x2, y2) = (landmarks[0][1], landmarks[0][2]), (landmarks[1][1], landmarks[1][2])
    cv2.circle(image, (x1, y1), 7, (0, 255, 0), cv2.FILLED)
    cv2.circle(image, (x2, y2), 7, (0, 255, 0), cv2.FILLED)
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)
    
    distance = hypot(x2 - x1, y2 - y1)
    return distance

if __name__ == '__main__':
    run()


Brightness set to: 28
Brightness set to: 48
Brightness set to: 51
Brightness set to: 50
Brightness set to: 61
Brightness set to: 62
Brightness set to: 62
Brightness set to: 63
Brightness set to: 63
Brightness set to: 64
Brightness set to: 67
Brightness set to: 68
Brightness set to: 72
Brightness set to: 72
Brightness set to: 73
Brightness set to: 76
Brightness set to: 74
Brightness set to: 61
Brightness set to: 55
Brightness set to: 48
Brightness set to: 31
Brightness set to: 28
Brightness set to: 14
Brightness set to: 11
Brightness set to: 7
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 0
Brightness set to: 2
Brightness set to: 21
Brightness set to: 25
Brightness set to: 29
Brightness set to: 43
Brightness set to: 46
Brightness set to: 49
Brightness set to: 52
Brightness set to: 59
Br

KeyboardInterrupt: 